In [ ]:
'''The following program opens a bunch of .csv files that are all in a folder
right now it opens one csv test file and finds the average temperature in degrees C 
from depths <=50 m for each year and returns a dictionary with years as keys and average 
temperatures as values'''

'''Things to do next:
1. Open all the csv files for a particular place one at a time
2. Plot the data of average temp vs. year with a line or scatter plot for each location
3. Make code into functions/better style'''

In [5]:
import csv
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import os
%matplotlib inline

# Opens the regional data .csv files from the designated folders and creates a list of the data.
# To read data you must organize relavent files into folders named for the region. Then all 
# region folders must be contained within a DataSets folder. Next you must extract the csv files
# from the data provided and delete the previous .csv.gz files. 
def read_in_regional_data(folder):
    regional_data_list = []
    directory = os.path.join("DataSets", folder)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        with open(os.path.join(directory, file), encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile) 
            for row in csv_reader:
                regional_data_list.append(row)
    return regional_data_list
regional_data_list = read_in_regional_data("SWIndianOcean")

<_io.TextIOWrapper name='DataSets\\SWIndianOcean\\ocldb1552588866.21219.CTD.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='DataSets\\SWIndianOcean\\ocldb1552588866.21219.MBT.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='DataSets\\SWIndianOcean\\ocldb1552588866.21219.MRB.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='DataSets\\SWIndianOcean\\ocldb1552588866.21219.OSD.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='DataSets\\SWIndianOcean\\ocldb1552588866.21219.PFL.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='DataSets\\SWIndianOcean\\ocldb1552588866.21219.SUR.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='DataSets\\SWIndianOcean\\ocldb1552588866.21219.XBT.csv' mode='r' encoding='utf-8'>


[['#--------------------------------------------------------------------------------',
  ''],
 ['CAST                        ',
  '',
  '        3240439',
  'WOD Unique Cast Number',
  'WOD code',
  ''],
 ['NODC Cruise ID              ', '', 'US-9433        ', '', '', ''],
 ['Originators Station ID      ',
  '',
  '              1',
  '',
  '',
  'integer',
  ''],
 ['Originators Cruise ID       ', '', '343427         ', '', '', ''],
 ['Latitude                    ',
  '',
  '        -13.158',
  'decimal degrees',
  '',
  ''],
 ['Longitude                   ',
  '',
  '        56.6083',
  'decimal degrees',
  '',
  ''],
 ['Year                        ', '', '           1974', '', '', ''],
 ['Month                       ', '', '              5', '', '', ''],
 ['Day                         ', '', '             19', '', '', ''],
 ['Time                        ',
  '',
  '           8.83',
  'decimal hours (UT)',
  '',
  ''],
 ['METADATA', ''],
 ['Country                     ',
  '',
  '   

In [6]:
# Reads the list row by row and takes out any spaces in individual strings due to the large amount of extra spacing in
# the data files. Returns the data file as a list with no extra spacing.
def regional_data_list_no_spaces(regional_data_list):
    data_file_list_no_spaces = []
    for row in regional_data_list:    
        new_row = []
        for word in row:
            new_word = word.replace(" ", "")
            new_row.append(new_word)
        data_file_list_no_spaces.append(new_row)
    return data_file_list_no_spaces
regional_data_list_no_spaces(regional_data_list)

NameError: name 'regional_data_list' is not defined

In [52]:
# Turns a list of lists into a new list of lists, where each inner list is a list of lists of all the data for
# each entry. Returns a list of lists for the data block.
def create_data_block_list_of_lists(data_file_list_no_spaces):
    data_block_list = []
    new_block = []
    for row in data_file_list_no_spaces:
        if "ENDOFVARIABLESSECTION" not in row:        
            new_block.append(row)        
        else:
            data_block_list.append(new_block)
            new_block = []
    return data_block_list
create_data_block_list_of_lists(data_file_list_no_spaces)

NameError: name 'data_file_list_no_spaces' is not defined

In [5]:
# Now make a list of dictionaries with the key of years, inner key of depths, and values of temperature
# Do the same for pH eventually
def create_year_and_temp_list(data_block_list):
    temperature_list = []
    for block in data_block_list:
        year_dict = {}
        depth_dict = {}
        for line in range(len(block)):       
            if 'Year' in block[line]:
                year = block[line][2]            
            if 'VARIABLES' and 'Temperatur' in block[line]:
                for i in range(line+3, len(block)):
                    depth = block[i][1]
                    temp = block[i][4]
                    if depth in depth_dict:
                        value = depth_dict[depth]
                        new_value = list(value).append(temp)
                        depth_dict[depth] = new_value
                    else:
                        depth_dict[depth] = temp
            year_dict[year] = depth_dict
        temperature_list.append(year_dict)
    return temperature_list
    #print(temperature_list)    

In [6]:
# Have a list of dictionaries with outer keys of years, and inner keys of depths, and inner values of temps
# Now take all dicts that have the same year and create a new dictionary that has years as keys and as values, 
# averages all of the temps together (for all depths less than or equal to 50)
def create_year_and_temp_dict(temperature_list):
    temp_dict = {}
    for row in temperature_list:
        year = list(row.keys())
        depth = list(row[year[0]].keys())
        temp = row[year[0]][depth[0]]
        if int(depth[0]) <= 50:
            if year[0] in temp_dict:
                temp_list = temp_dict[year[0]]
                temp_list.append(temp)
            else:
                temp_dict[year[0]] = [temp]
    return temp_dict
    #print(temp_dict)    

In [7]:
# Now average together all the temperature values for each year key
def create_year_and_avg_temp_dict(temp_dict):
    avg_temp_dict = {}
    for year in temp_dict:
        temps = temp_dict[year]
        total_number_temps = len(temps)
        total_temp = 0
        for temp in temps:
            total_temp += int(temp)
        avg_temp = total_temp/total_number_temps
        avg_temp_dict[year] = avg_temp
    return avg_temp_dict
    print(avg_temp_dict)

In [10]:
def create_lineplot_of_temp_data(avg_temp_dict):
    for value in avg_temp_dict:
        year_list = []
        index_list = []
        for year in avg_temp_dict[value]:
            year_list.append(year)
            index_list.append(avg_temp_dict[value][year])
        plt.plot(year_list, index_list, 'o', label = value)
        plt.legend(loc=9, bbox_to_anchor=(0.5, -0.1))

create_lineplot_of_temp_data(avg_temp_dict)    
plt.show()  

TypeError: create_lineplot_of_temp_data() missing 1 required positional argument: 'avg_temp_dict'

In [ ]:
# Where region is either "SWIndianOcean", "CarribeanSea", or "Australia"
# for the scope of this course. 
def main(region):
    read_in_regional_data(region)
    regional_data_list_no_spaces(regional_data_list)
    create_data_block_list_of_lists(data_file_list_no_spaces)
    create_year_and_temp_list(data_block_list)
    create_year_and_temp_dict(temperature_list)
    create_year_and_avg_temp_dict(temp_dict)

main()
#     read_in_regional_data("CarribeanSea")
#     regional_data_list_no_spaces(regional_data_list)
#     create_data_block_list_of_lists(data_file_list_no_spaces)
#     create_year_and_temp_list(data_block_list)
#     create_year_and_temp_dict(temperature_list)
#     create_year_and_avg_temp_dict(temp_dict)
    
#     read_in_regional_data("Australia")
#     regional_data_list_no_spaces(regional_data_list)
#     create_data_block_list_of_lists(data_file_list_no_spaces)
#     create_year_and_temp_list(data_block_list)
#     create_year_and_temp_dict(temperature_list)
#     create_year_and_avg_temp_dict(temp_dict)